# ML Workflow

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
import requests
import xmltodict

### 1. GET THE DATA

In [18]:
df = pd.read_csv("cryptoranktable_220410.csv", sep = ";", decimal = ",")

In [41]:
df.head(30)

,Name,Price,Sale Price,Market Cap,Raise,ROI USD,ATH ROI USD,Type,TGE Platform,Ended,Category,ticker
0,EdgeSwapEGS,0.1170,0.0150,6640000.0,150.000,7.81,15.75,IEO,Huobi Global,2022-04-07,Exchange,EGS
1,EdgeSwapEGS,0.1170,0.0150,6640000.0,70.000,7.81,15.75,IEO,Gate.io,2022-04-07,Exchange,EGS
2,Drunk RobotsMETAL,0.0559,0.0100,1050000.0,150.000,5.59,6.90,IEO,Gate.io,2022-04-07,Gaming,METAL
3,WinerzWNZ,NaN,0.0200,NaN,500.000,NaN,NaN,IDO,SolRazr,2022-04-07,Gaming,WNZ
4,WinerzWNZ,NaN,0.0200,NaN,500.000,NaN,NaN,IDO,NaN,2022-04-07,Gaming,WNZ
5,MoonwellWELL,NaN,0.0260,NaN,13.000.000,NaN,NaN,ICO,NaN,2022-04-07,DeFi,WELL
6,Drunk RobotsMETAL,0.0559,0.0100,1050000.0,200.000,5.59,6.90,IDO,GameFi,2022-04-06,Gaming,METAL
7,Drunk RobotsMETAL,0.0559,0.0100,1050000.0,50.000,5.59,6.90,IDO,TrustPad,2022-04-06,Gaming,METAL
8,STREETHSTREETH,0.0824,0.0700,503990.0,200.000,1.18,2.06,IDO,BinStarter,2022-04-05,Non-Fungible,TREETH
9,BlueshiftBLUES,NaN,0.2500,NaN,250.000,NaN,NaN,IDO,OccamRazer,2022-04-05,DeFi,BLUES


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          3254 non-null   object 
 1   Price         3118 non-null   float64
 2   Sale Price    3223 non-null   float64
 3   Market Cap    2835 non-null   float64
 4   Raise         2941 non-null   object 
 5   ROI USD       3092 non-null   float64
 6   ATH ROI USD   3092 non-null   float64
 7   Type          3254 non-null   object 
 8   TGE Platform  2203 non-null   object 
 9   Ended         3163 non-null   object 
 10  Category      3171 non-null   object 
dtypes: float64(5), object(6)
memory usage: 279.8+ KB


In [22]:
df['ticker'] = df['Name'].apply(getTicker)

Get the URLs of the coins, names and slugs

In [90]:
# some urls of coinrank
url_coins = 'https://cryptorank.io/sitemap-coins.xml'

response = requests.get(url_coins)
urlcoins_data = xmltodict.parse(response.content)

In [94]:
url_list = []
for url in urlcoins_data['urlset']['url']:
    url_list.append(url['loc'])

In [177]:
url_list[38].rsplit('/')

['https:', '', 'cryptorank.io', 'price', 'w3coin', 'arbitrage']

In [224]:
coin_names = []
for row in url_list:
    if row.rsplit('/')[3] == 'ico':
        coin_names.append(row.rsplit('/')[4])

Get the data from request /coins/{id}

In [226]:
request_coins = 'https://api.coingecko.com/api/v3/coins/gamium'

response = requests.get(request_coins).json()

In [236]:
coin_data = {}
# symbols and names
coin_data['id'] = response['id']
coin_data['symbol'] = response['symbol']
coin_data['name'] = response['name']
# platform, categories and coin info
coin_data['asset_platform_id'] = response['asset_platform_id']
coin_data['block_time_in_minutes'] = response['block_time_in_minutes']
coin_data['hashing_algorithm'] = response['hashing_algorithm']
coin_data['categories'] = response['categories']
coin_data['public_notice'] = response['public_notice']
coin_data['additional_notices'] = response['additional_notices']
coin_data['description'] = response['description']['en']
coin_data['country_origin'] = response['country_origin']
coin_data['genesis_date'] = response['genesis_date']
#sentiment and scores
coin_data['sentiment_votes_up_percentage'] = response['sentiment_votes_up_percentage']
coin_data['sentiment_votes_down_percentage'] = response['sentiment_votes_down_percentage']
coin_data['market_cap_rank'] = response['market_cap_rank']
coin_data['coingecko_rank'] = response['coingecko_rank']
coin_data['coingecko_score'] = response['coingecko_score']
coin_data['developer_score'] = response['developer_score']
coin_data['community_score'] = response['community_score']
coin_data['liquidity_score'] = response['liquidity_score']
coin_data['public_interest_score'] = response['public_interest_score']
#market data
coin_data['total_value_locked'] = response['total_value_locked']
coin_data['mcap_to_tvl_ratio'] = response['mcap_to_tvl_ratio']
coin_data['fdv_to_tvl_ratio'] = response['fdv_to_tvl_ratio']
coin_data['roi'] = response['roi']
#market data - all time high data
coin_data['ath_usd'] = response['ath']['usd']
coin_data['ath_btc'] = response['ath']['btc']
coin_data['ath_eth'] = response['ath']['eth']
coin_data['ath_change_percentage_usd'] = response['ath_change_percentage']['usd']
coin_data['ath_change_percentage_btc'] = response['ath_change_percentage']['btc']
coin_data['ath_change_percentage_eth'] = response['ath_change_percentage']['eth']
coin_data['ath_date_usd'] = response['ath_date']['usd']
coin_data['ath_date_btc'] = response['ath_date']['btc']
coin_data['ath_date_eth'] = response['ath_date']['eth']
#market data - all time low data
coin_data['atl_usd'] = response['market_data']['atl']['usd']
coin_data['atl_btc'] = response['market_data']['atl']['btc']
coin_data['atl_eth'] = response['market_data']['atl']['eth']
coin_data['atl_change_percentage_usd'] = response['market_data']['atl_change_percentage']['usd']
coin_data['atl_change_percentage_btc'] = response['market_data']['atl_change_percentage']['btc']
coin_data['atl_change_percentage_eth'] = response['market_data']['atl_change_percentage']['eth']
coin_data['atl_date_usd'] = response['market_data']['atl_date']['usd']
coin_data['atl_date_btc'] = response['market_data']['atl_date']['btc']
coin_data['atl_date_eth'] = response['market_data']['atl_date']['eth']
#market data - market cap data
coin_data['market_cap_usd'] = response['market_data']['market_cap']['usd']
coin_data['market_cap_btc'] = response['market_data']['market_cap']['btc']
coin_data['market_cap_eth'] = response['market_data']['market_cap']['eth']
coin_data['market_cap_rank'] = response['market_data']['market_cap_rank']
#community data
coin_data['facebook_likes'] = response['community_data']['facebook_likes']
coin_data['twitter_followers'] = response['community_data']['twitter_followers']
coin_data['reddit_average_posts_48h'] = response['community_data']['reddit_average_posts_48h']
coin_data['reddit_average_comments_48h'] = response['community_data']['reddit_average_comments_48h']
coin_data['reddit_subscribers'] = response['community_data']['reddit_subscribers']
coin_data['reddit_accounts_active_48h'] = response['community_data']['reddit_accounts_active_48h']
coin_data['telegram_channel_user_count'] = response['community_data']['telegram_channel_user_count']
#developers data
coin_data['developer_data'] = response['developer_data']
coin_data['public_interest_stats'] = response['public_interest_stats']


KeyError: 'total_value_locked'

In [232]:
coin_data = {}
coin_data

{}

In [220]:
for coin in coin_names:
    request_coin = f'https://api.coingecko.com/api/v3/coins/{coin}'
    response.json()['description']['en']

'Gamium is building the first decentralized social metaverse and the digital identity of humans.\r\nCreate your avatar, exchange NFTs, get a job, socialize and earn in the Gamium world'

In [225]:
coins_dict = {}
cd = pd.json_normalize(response.json())
cd

,id,symbol,name,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,public_notice,additional_notices,country_origin,...,developer_data.total_issues,developer_data.closed_issues,developer_data.pull_requests_merged,developer_data.pull_request_contributors,developer_data.code_additions_deletions_4_weeks.additions,developer_data.code_additions_deletions_4_weeks.deletions,developer_data.commit_count_4_weeks,developer_data.last_4_weeks_commit_activity_series,public_interest_stats.alexa_rank,public_interest_stats.bing_matches
0,gamium,gmm,Gamium,binance-smart-chain,0,None,"[DaoMaker Ecosystem, Binance Smart Chain Ecosy...",None,[],,...,0,0,0,0,138,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",None,None


In [197]:
coins_dict

{'id': 0    gamium
 Name: id, dtype: object,
 'symbol': 0    gmm
 Name: symbol, dtype: object,
 'name': 0    Gamium
 Name: name, dtype: object,
 'asset_platform_id': 0    binance-smart-chain
 Name: asset_platform_id, dtype: object,
 'block_time_in_minutes': 0    0
 Name: block_time_in_minutes, dtype: int64,
 'hashing_algorithm': 0    None
 Name: hashing_algorithm, dtype: object}

### 2. Clean the Data

In [38]:
# Duplicate -> Sabemos que tem muitos projetos duplicados com os mesmos numeros em todos! Sendo a unica diferenca a plataforma lancada
# Existem 2210 projetos diferentes na base de dados com 3254 IDOs/ICOs/IEOs
# 1593 projetos fizeram somente um IDO, ou um ICO, ou um IEO - ou tambem pode ser que fizeram um IDO, e apos um tempo um IEO para arrecadar mais capital e entrar num exchange grande
# 617 projetos fizeram o IDO em diferentes plataformas, no geral finalizando sempre no mesmo dia. o maximo foi o projeto Solschick com o IDO em 15 plataformas diferentes, mas a media tende a ser muito menor (1,4), sendo a mediana igual a 1 

In [40]:
df.isnull().sum().sort_values(ascending=False)/len(df) #NaN percentage for each column

TGE Platform    0.322987
Market Cap      0.128765
Raise           0.096189
ROI USD         0.049785
ATH ROI USD     0.049785
Price           0.041795
Ended           0.027966
Category        0.025507
Sale Price      0.009527
Name            0.000000
Type            0.000000
ticker          0.000000
dtype: float64